<a href="https://colab.research.google.com/github/vellos0x1/Demz1/blob/main/Gading_Kirana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install segment-geospatial leafmap localtileserver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 564.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [4]:
import os
import leafmap
import torch
from samgeo import SamGeo, tms_to_geotiff

In [5]:
torch.cuda.get_device_name(0)


'Tesla T4'

In [7]:
m = leafmap.Map(center=[-6.143976661737807, 106.8981213332354], zoom=19)
m.add_basemap('SATELLITE')
m

Map(center=[-6.143976661737807, 106.8981213332354], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [9]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()

In [10]:
image = 'satellite.tif'

In [11]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source='Satellite')

Downloaded image 1/504
Downloaded image 2/504
Downloaded image 3/504
Downloaded image 4/504
Downloaded image 5/504
Downloaded image 6/504
Downloaded image 7/504
Downloaded image 8/504
Downloaded image 9/504
Downloaded image 10/504
Downloaded image 11/504
Downloaded image 12/504
Downloaded image 13/504
Downloaded image 14/504
Downloaded image 15/504
Downloaded image 16/504
Downloaded image 17/504
Downloaded image 18/504
Downloaded image 19/504
Downloaded image 20/504
Downloaded image 21/504
Downloaded image 22/504
Downloaded image 23/504
Downloaded image 24/504
Downloaded image 25/504
Downloaded image 26/504
Downloaded image 27/504
Downloaded image 28/504
Downloaded image 29/504
Downloaded image 30/504
Downloaded image 31/504
Downloaded image 32/504
Downloaded image 33/504
Downloaded image 34/504
Downloaded image 35/504
Downloaded image 36/504
Downloaded image 37/504
Downloaded image 38/504
Downloaded image 39/504
Downloaded image 40/504
Downloaded image 41/504
Downloaded image 42/504
D

In [12]:
m.add_raster(image, layer_name='Image')
m

Map(bottom=17351384.80001831, center=[-6.145128892295249, 106.89984048402683], controls=(ZoomControl(options=[…

In [13]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
checkpoint = os.path.join(out_dir, 'sam_vit_h_4b8939.pth')

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(
    checkpoint=checkpoint,
    model_type='vit_h',
    device=device,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs=None,
)

Checkpoint /root/Downloads/sam_vit_h_4b8939.pth does not exist.


Downloading...
From: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
To: /root/Downloads/sam_vit_h_4b8939.pth
100%|██████████| 2.56G/2.56G [00:11<00:00, 232MB/s]


In [16]:
mask = 'segment.tiff'
sam.generate(image, mask)

100%|██████████| 132/132 [14:54<00:00,  6.78s/it]


In [18]:
shapefile = 'segment.shp'
sam.tiff_to_vector(mask, shapefile)

In [23]:
style = {
    'color': '#3388ff',
    'weight': 2,
    'fillColor': '#7c4185',
    'fillOpacity': 0.5,
}

In [30]:
style = {
    'color': '#3388ff',
    'weight': 2,
    'fillColor': '#7c4185',
    'fillOpacity': 0.5,
}
m.add_vector(vector, layer_name='vector', style=style)
m

NameError: ignored